In [1]:
import torch
from tqdm.notebook import tqdm
import os
from transformers import BertTokenizer, DistilBertTokenizer, TFXLNetModel, XLNetTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset
import numpy as np
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Project_News_Classification')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Mounted at /content/drive


device(type='cuda', index=0)

In [2]:
# Specify the path to your JSON file
json_path = 'nytimes.json'
# Read the JSON data into a DataFrame
df = pd.read_json(json_path)
#df = df.sample(5000)
df

,section,headline,article_url,article,abstract,article_id,image,caption,image_id
0,Health,F.D.A. Plans to Ban Most E-Cigarette Flavors b...,https://www.nytimes.com/2019/12/31/health/e-ci...,The Trump administration is expected to announ...,The tobacco and vaping industries and conserva...,42d25485-0e48-50bf-8d16-948833b2a55d,https://static01.nyt.com/images/2019/11/06/sci...,A new study by the National Institute on Drug ...,42d25485-0e48-50bf-8d16-948833b2a55d
1,Science,Meteor Showers in 2020 That Will Light Up Nigh...,https://www.nytimes.com/2020/01/01/science/met...,All year long as Earth revolves around the sun...,"All year long, Earth passes through streams of...",04bc90f0-b20b-511c-b5bb-3ce13194163f,https://static01.nyt.com/images/2020/01/01/sci...,"Perseid meteors named as ""Orinoid"" streak acro...",04bc90f0-b20b-511c-b5bb-3ce13194163f
2,Science,"Rocket Launches, Trips to Mars and More 2020 S...",https://www.nytimes.com/2020/01/01/science/spa...,"If you follow space news and astronomy, the pa...",A year full of highs and lows in space just en...,bd8647b3-8ec6-50aa-95cf-2b81ed12d2dd,https://static01.nyt.com/images/2020/01/01/sci...,Spectators viewing the launch of a Soyuz rocke...,bd8647b3-8ec6-50aa-95cf-2b81ed12d2dd
3,Television,What's on TV Wednesday: A Linda Ronstadt Doc a...,https://www.nytimes.com/2020/01/01/arts/televi...,LINDA RONSTADT: THE SOUND OF MY VOICE (2019) 9...,"""Linda Ronstadt: The Sound of My Voice"" airs o...",e6c25b53-0416-5795-b0cf-e1243924dc79,https://static01.nyt.com/images/2020/01/01/art...,"Linda Ronstadt in ""Linda Ronstadt: The Sound o...",e6c25b53-0416-5795-b0cf-e1243924dc79
4,Travel,New Cruise Ships to Set Sail for Antarctica,https://www.nytimes.com/2020/01/01/travel/anta...,As the number of travelers cruising Antarctica...,Interested in the southernmost continent? Here...,98c3d182-95ce-5244-9b9e-008a3dee7354,https://static01.nyt.com/images/2020/01/05/tra...,"Antarctica21&rsquo;s expedition ship, Ocean No...",98c3d182-95ce-5244-9b9e-008a3dee7354
...,...,...,...,...,...,...,...,...,...
61230,Global Business,"For Sushi Chain, Conveyor Belts Carry Profit",https://www.nytimes.com/2010/12/31/business/gl...,"SAYAMA, Japan The Kura ""revolving sushi"" rest...","The Kura ""revolving sushi"" chain has succeeded...",a3f8bf67-1938-5d2b-8d83-6e7edb80d5c0,https://static01.nyt.com/images/2010/12/31/bus...,"Kura, a chain in Japan, relies on small staffs...",a3f8bf67-1938-5d2b-8d83-6e7edb80d5c0
61231,Automobiles,Clouds Parted to a Forecast of Sunnier Days,https://www.nytimes.com/2011/01/02/automobiles...,THE last year in automobiles could have been m...,"For the automotive industry, 2010 played out l...",75199d98-0c30-51b9-8efa-4783c078260e,https://static01.nyt.com/images/2011/01/02/aut...,"DEBUTANTES Clockwise from far left, a few news...",75199d98-0c30-51b9-8efa-4783c078260e
61232,Automobiles,Jerry Garrett's Top 10 Cars of 2010,https://www.nytimes.com/2011/01/02/automobiles...,1. ASTON MARTIN RAPIDE The Rapide is something...,From a sleek European sedan to the last effort...,8af1e9f0-2fb8-57f1-9a10-25ca06743306,https://static01.nyt.com/images/2011/01/02/aut...,Aston Martin Rapide.,8af1e9f0-2fb8-57f1-9a10-25ca06743306
61233,Automobiles,Lawrence Ulrich's Top 10 Cars of 2010,https://www.nytimes.com/2011/01/02/automobiles...,1. FERRARI 458 ITALIA The world's best sports ...,From the world's best sports car to an unexpec...,cfab59b0-2357-54b6-a469-7d5574d60e9c,https://static01.nyt.com/images/2011/01/02/aut...,Ferrari 458 Italia.,cfab59b0-2357-54b6-a469-7d5574d60e9c


In [3]:
# Remove the columns
df = df.drop(columns=['headline', 'abstract', 'article_url', 'article_id', 'image', 'caption', 'image_id'], axis=1)
df.head()

,section,article
0,Health,The Trump administration is expected to announ...
1,Science,All year long as Earth revolves around the sun...
2,Science,"If you follow space news and astronomy, the pa..."
3,Television,LINDA RONSTADT: THE SOUND OF MY VOICE (2019) 9...
4,Travel,As the number of travelers cruising Antarctica...


In [4]:
df['section'].value_counts()

section
Health             3001
Opinion            3001
Technology         3001
Fashion & Style    3001
Food               3001
Media              3001
Art & Design       3001
Science            3001
Music              3001
Books              3001
Theater            3001
Sports             3001
Real Estate        3001
Dance              3001
Movies             3001
Travel             3001
Television         3001
Style              2681
Automobiles        1825
Economy            1761
Your Money         1263
Global Business    1182
Education           825
Well                681
Name: count, dtype: int64

In [5]:
possible_labels = df.section.unique()
label_dict = {}
id2label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
    id2label_dict[index] = possible_label
print(label_dict)

{'Health': 0, 'Science': 1, 'Television': 2, 'Travel': 3, 'Movies': 4, 'Dance': 5, 'Real Estate': 6, 'Economy': 7, 'Sports': 8, 'Theater': 9, 'Opinion': 10, 'Music': 11, 'Books': 12, 'Art & Design': 13, 'Style': 14, 'Media': 15, 'Food': 16, 'Well': 17, 'Fashion & Style': 18, 'Technology': 19, 'Your Money': 20, 'Education': 21, 'Automobiles': 22, 'Global Business': 23}


In [6]:
print(id2label_dict)

{0: 'Health', 1: 'Science', 2: 'Television', 3: 'Travel', 4: 'Movies', 5: 'Dance', 6: 'Real Estate', 7: 'Economy', 8: 'Sports', 9: 'Theater', 10: 'Opinion', 11: 'Music', 12: 'Books', 13: 'Art & Design', 14: 'Style', 15: 'Media', 16: 'Food', 17: 'Well', 18: 'Fashion & Style', 19: 'Technology', 20: 'Your Money', 21: 'Education', 22: 'Automobiles', 23: 'Global Business'}


In [7]:
df['label'] = df.section.replace(label_dict)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.2,
                                                  random_state=2024,
                                                  stratify=df.label.values)

In [9]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [10]:
df.groupby(['section', 'label', 'data_type']).count()

article
section         label data_type         
Art & Design    13    train         2401
                      val            600
Automobiles     22    train         1460
                      val            365
Books           12    train         2400
                      val            601
Dance           5     train         2401
                      val            600
Economy         7     train         1409
                      val            352
Education       21    train          660
                      val            165
Fashion & Style 18    train         2401
                      val            600
Food            16    train         2401
                      val            600
Global Business 23    train          945
                      val            237
Health          0     train         2400
                      val            601
Media           15    train         2401
                      val            600
Movies          4     train         2401
                      val            600
Music           11    train         2401
                      val            600
Opinion         10    train         2401
                      val            600
Real Estate     6     train         2401
                      val            600
Science         1     train         2401
                      val            600
Sports          8     train         2401
                      val            600
Style           14    train         2145
                      val            536
Technology      19    train         2400
                      val            601
Television      2     train         2401
                      val            600
Theater         9     train         2401
                      val            600
Travel          3     train         2401
                      val            600
Well            17    train          545
                      val            136
Your Money      20    train         1010
                      val            253

In [11]:
xlnet_model = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(xlnet_model, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [12]:
from torch.utils.data import Dataset, DataLoader

In [13]:
class N24Dataset(Dataset):

    def __init__(self, article, label, tokenizer, max_len):
        self.article = article
        self.label = label
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.article)

    def __getitem__(self, item):
        article = str(self.article[item])
        label = self.label[item]

        encoding = self.tokenizer.encode_plus(
        article,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=self.max_len, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids)

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=self.max_len, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)

        return {
        'article_text': article,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(label, dtype=torch.long)
        }

In [14]:
df_train, df_val = train_test_split(df, test_size=0.2, random_state=2024)

In [15]:
df_val

,section,article,label,data_type
38542,Dance,Researchers have found that over half the worl...,5,train
11394,Books,"""In the raging battle between the president an...",12,val
38425,Technology,If you see your family pictures starting to fa...,19,train
37936,Books,AN ALMOST PERFECT CHRISTMAS \n\nBy Nina Stibbe...,12,train
22830,Fashion & Style,Credit...Charlotte Hadden for The New York Tim...,18,train
...,...,...,...,...
6366,Opinion,"In 1988, my family and I were honored when New...",10,train
7550,Opinion,Our democracy hangs in the balance. This is no...,10,train
14900,Opinion,Even as the campaign lawsuits brought by Presi...,10,train
46101,Travel,"From about 1,755 Hong Kong dollars (or about $...",3,train


In [16]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = N24Dataset(
    article=df.article.to_numpy(),
    label=df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len

  )

  return DataLoader(
    ds,
    batch_size=batch_size
  )

In [17]:
BATCH_SIZE = 2
MAX_LEN = 512
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 24)
model = model.to(device)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
EPOCHS = 3

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
data = next(iter(val_data_loader))
data.keys()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


dict_keys(['article_text', 'input_ids', 'attention_mask', 'targets'])

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
targets = data['targets'].to(device)
print(input_ids.reshape(-1,512).shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([2, 512])
torch.Size([2, 512])


In [ ]:
input_ids.reshape(-1,512)

tensor([[ 3546,    47,   255,  ...,    38,     4,     3],
        [   17,    12,   153,  ..., 18610,     4,     3]], device='cuda:0')

In [ ]:
outputs = model(input_ids.reshape(-1,512), token_type_ids=None, attention_mask=attention_mask, labels=targets)
outputs

XLNetForSequenceClassificationOutput(loss=tensor(3.3308, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.5406, -0.0743,  0.1998,  0.2867, -0.0069,  0.3644,  0.4577, -0.1645,
         -0.0475, -0.1079, -0.0344,  0.1575, -0.3367,  0.1513, -0.0044,  0.2698,
          0.0654, -0.0537, -0.0280,  0.1078, -0.8226,  0.5629,  0.1955, -0.2736],
        [-0.0604, -0.1187,  0.7394, -0.0336,  0.1186,  0.5606,  0.2621, -0.2474,
         -0.1397,  0.3010, -0.1781,  0.2382, -0.5357,  0.0700,  0.1436,  0.2161,
          0.3431,  0.0475,  0.1991,  0.3202, -0.4441, -0.1673, -0.2531, -0.8778]],
       device='cuda:0', grad_fn=<AddmmBackward0>), mems=(tensor([[[-0.0371,  0.0284, -0.0004,  ..., -0.0005, -0.0954, -0.0588],
         [-0.0387, -0.0099, -0.0053,  ...,  0.0643,  0.0192, -0.0144]],

        [[-0.0324, -0.0562,  0.0424,  ...,  0.0218,  0.0194, -0.0013],
         [-0.0460, -0.0333, -0.0153,  ..., -0.0029,  0.0316, -0.0187]],

        [[ 0.0881, -0.0523,  0.0990,  ...,  0.0669, -0.0

In [ ]:
for d in val_data_loader:
    test_input_ids = d["input_ids"].reshape(-1,512).to(device)

In [ ]:
from sklearn import metrics
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0

    progress_bar = tqdm(data_loader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for d in progress_bar:
        input_ids = d["input_ids"].reshape(-1,512).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())

        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].reshape(-1,512).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1
            y_true.extend(targets)
            y_pred.extend(prediction)

    recall = metrics.recall_score(y_true, y_pred, average='weighted')
    precision = metrics.precision_score(y_true, y_pred, average='weighted')
    f1_score = metrics.f1_score(y_true, y_pred, average='weighted')

    return acc / counter, np.mean(losses), recall, precision, f1_score

In [ ]:
from collections import defaultdict
from torch import nn, optim
history = defaultdict(list)
best_accuracy = 0

for epoch in tqdm(range(1, EPOCHS+1)):
    print(f'Epoch {epoch}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        optimizer,
        device,
        scheduler,
        len(df_train)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        device,
        len(df_val)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'xlnet_model.bin')
        best_accuracy = val_acc

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/3
----------


Epoch 1:   0%|          | 0/24494 [00:00<?, ?it/s]

Train loss 1.0159913747170626 Train accuracy 0.7986445660161672
Val loss 0.9190280766515849 Val accuracy 0.8361365120836055

Epoch 2/3
----------


Epoch 2:   0%|          | 0/24494 [00:00<?, ?it/s]

Train loss 0.6480683392272139 Train accuracy 0.8796031681228056
Val loss 0.8133412701379112 Val accuracy 0.8633246244284781

Epoch 3/3
----------


Epoch 3:   0%|          | 0/24494 [00:00<?, ?it/s]

Train loss 0.40054265821308105 Train accuracy 0.927104597044174
Val loss 0.7733845276269754 Val accuracy 0.8769595035924233



In [ ]:
EPOCHS = 5

for epoch in tqdm(range(4, EPOCHS+1)):
    print(f'Epoch {epoch}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        optimizer,
        device,
        scheduler,
        len(df_train)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        device,
        len(df_val)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'xlnet_model.bin')
        best_accuracy = val_acc

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 4/5
----------


Epoch 4:   0%|          | 0/24494 [00:00<?, ?it/s]

Train loss 0.2632720358257196 Train accuracy 0.9495386625295991
Val loss 0.7733845276269754 Val accuracy 0.8769595035924233

Epoch 5/5
----------


Epoch 5:   0%|          | 0/24494 [00:00<?, ?it/s]

Train loss 0.26299587693399507 Train accuracy 0.9498856862905202
Val loss 0.7733845276269754 Val accuracy 0.8769595035924233



In [18]:
from transformers import XLNetForSequenceClassification, XLNetTokenizer

xlnet_model = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(xlnet_model, do_lower_case=True)


model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 24)
model = model.to(device)

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.load_state_dict(torch.load('xlnet_model.bin', map_location=torch.device('cpu')))

<All keys matched successfully>

In [21]:
import numpy as np
from sklearn import metrics
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].reshape(-1,512).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1
            y_true.extend(targets)
            y_pred.extend(prediction)

    recall = metrics.recall_score(y_true, y_pred, average='weighted')
    precision = metrics.precision_score(y_true, y_pred, average='weighted')
    f1_score = metrics.f1_score(y_true, y_pred, average='weighted')

    return acc / counter, np.mean(losses), recall, precision, f1_score

In [22]:
val_acc, val_loss, val_recall, val_precision, val_f1score = eval_model(model,
        val_data_loader,
        device,
        len(df_val))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [23]:
print("accuracy:",val_acc)
print("recall:",val_recall)
print("precision:",val_precision)
print("f1:",val_f1score)

accuracy: 0.8769595035924233
recall: 0.8769494570098799
precision: 0.8748375467741367
f1: 0.875491822465624
